<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import statistics
import numpy as np
import sys
from graphviz import Digraph

In [2]:
class Node:
    def __init__(self, val, weight, axis=0, minWeight=0, maxWeight=0, count=0):
        self.right = self.left = None
        self.val = val
        self.weight = weight
        self.axis = axis
        self.minWeight = minWeight
        self.maxWeight = maxWeight
        self.count = count

In [3]:
def preorderTraversal(root):
    res, stack = [], [root]
    while stack:
        node = stack.pop()
        if node:
            res.insert(0, [node.val,node.weight])
            stack.append(node.left)
            stack.append(node.right)
    return res

In [59]:
def dfs(root):
    res, stack = {}, [root]
    while stack:
        for i in range(len(stack)):
            node = stack.pop(0)
            if node:
                stack.append(node.left)
                stack.append(node.right)
                res[str([node.val, node.weight, node.minWeight, node.maxWeight, node.count])] = []
                if node.left:
                    res[str([node.val, node.weight, node.minWeight, node.maxWeight, node.count])].append(str([node.left.val, node.left.weight, node.left.minWeight, node.left.maxWeight, node.left.count]))
                if node.right:
                    res[str([node.val, node.weight, node.minWeight, node.maxWeight, node.count])].append(str([node.right.val, node.right.weight, node.right.minWeight, node.right.maxWeight, node.right.count]))
                print("root: ", [node.val, node.weight] if node else "None", "left: ", [node.left.val, node.left.weight] if node.left else "None", "right: ", [node.right.val, node.right.weight] if node.right else "None", "axis: ", "None" if node.axis == "None" else node.axis)
                print("weight: ", [node.minWeight, node.maxWeight, node.count] if node else "None")
        print(res)
    return res

In [60]:
def buildKdTree(points, hyperplaneAxis = 0, weights = None):
    if weights == None:
        points = [[point] for point in points]
    else:
        if len(points) != len(weights):
            print("Incompatible len of points and weights")
            return
        points = [[point, weight] for point, weight in zip(points, weights)]
    return buildTreeWithMedian(points, hyperplaneAxis)

In [61]:
def buildTreeWithMedian(points, hyperplaneAxis = 0):
    
    dim = len(points[0][0])
    nxtHyperplaneAxis = (hyperplaneAxis + 1) % dim
    
    if len(points) == 1:
#         print("when points are 1: ", points[0][0], points[0][1])
        return Node(points[0][0], points[0][1], nxtHyperplaneAxis, points[0][1], points[0][1], 1)
    
    medianIndex = len(points) // 2
    sortedPoints = sorted(points, key=lambda x: x[0][hyperplaneAxis])

    median = sortedPoints[medianIndex]
    leftPoints = sortedPoints[:medianIndex]
    rightPoints = sortedPoints[medianIndex + 1:]

    sortedWeights = sorted(points, key=lambda x: x[1])
#     print(sortedWeights[0][1])
#     print("sorted based on: ", dim , sortedPoints)
#     print("median: ", median)
#     print("leftPoints: ", leftPoints)
#     print("rightPoints: ", rightPoints)
#     print("hyperplane axis: ", hyperplaneAxis)
    
    currNode = Node(median[0], median[1], hyperplaneAxis, sortedWeights[0][1], sortedWeights[-1][1], len(sortedWeights))
    currNode.left = buildTreeWithMedian(leftPoints, nxtHyperplaneAxis) if len(leftPoints) > 0 else []
    currNode.right = buildTreeWithMedian(rightPoints, nxtHyperplaneAxis) if len(rightPoints) > 0 else []
#     print("root: ", [currNode.val, currNode.weight] if currNode else "None", "left: ", [currNode.left.val, currNode.left.weight] if currNode.left else "None", "right: ", [currNode.right.val, currNode.right.weight] if currNode.right else "None", "axis: ", "None" if currNode.axis == "None" else currNode.axis)    
    return currNode

In [62]:
points = np.array([(86, 338), (164, 360), (75, 58), (5,358), (400, 346), (281, 411), (136, 39), (324, 54),(296,332)])

In [63]:
# points = np.array([(86, 338, 13), (164, 360, 34), (75, 58, 190), (5,358, 120),(400, 346, 290), (281, 411, 342), (136, 39, 456),(324, 54, 100),(296,332, 230)])

In [64]:
weights = [10, 5, 20, 15, 30, 18, 4, 23, 29]

In [65]:
root = buildKdTree(points,0,weights)

In [66]:
preorder = preorderTraversal(root)

In [67]:
adjacencyList = dfs(root)

root:  [array([164, 360]), 5] left:  [array([ 86, 338]), 10] right:  [array([400, 346]), 30] axis:  0
weight:  [4, 30, 9]
{'[array([164, 360]), 5, 4, 30, 9]': ['[array([ 86, 338]), 10, 4, 20, 4]', '[array([400, 346]), 30, 18, 30, 4]']}
root:  [array([ 86, 338]), 10] left:  [array([136,  39]), 4] right:  [array([  5, 358]), 15] axis:  1
weight:  [4, 20, 4]
root:  [array([400, 346]), 30] left:  [array([324,  54]), 23] right:  [array([281, 411]), 18] axis:  1
weight:  [18, 30, 4]
{'[array([164, 360]), 5, 4, 30, 9]': ['[array([ 86, 338]), 10, 4, 20, 4]', '[array([400, 346]), 30, 18, 30, 4]'], '[array([ 86, 338]), 10, 4, 20, 4]': ['[array([136,  39]), 4, 4, 20, 2]', '[array([  5, 358]), 15, 15, 15, 1]'], '[array([400, 346]), 30, 18, 30, 4]': ['[array([324,  54]), 23, 23, 29, 2]', '[array([281, 411]), 18, 18, 18, 1]']}
root:  [array([136,  39]), 4] left:  [array([75, 58]), 20] right:  None axis:  0
weight:  [4, 20, 2]
root:  [array([  5, 358]), 15] left:  None right:  None axis:  1
weight:  

In [68]:
adjacencyList

{'[array([164, 360]), 5, 4, 30, 9]': ['[array([ 86, 338]), 10, 4, 20, 4]',
  '[array([400, 346]), 30, 18, 30, 4]'],
 '[array([ 86, 338]), 10, 4, 20, 4]': ['[array([136,  39]), 4, 4, 20, 2]',
  '[array([  5, 358]), 15, 15, 15, 1]'],
 '[array([400, 346]), 30, 18, 30, 4]': ['[array([324,  54]), 23, 23, 29, 2]',
  '[array([281, 411]), 18, 18, 18, 1]'],
 '[array([136,  39]), 4, 4, 20, 2]': ['[array([75, 58]), 20, 20, 20, 1]'],
 '[array([  5, 358]), 15, 15, 15, 1]': [],
 '[array([324,  54]), 23, 23, 29, 2]': ['[array([296, 332]), 29, 29, 29, 1]'],
 '[array([281, 411]), 18, 18, 18, 1]': [],
 '[array([75, 58]), 20, 20, 20, 1]': [],
 '[array([296, 332]), 29, 29, 29, 1]': []}

In [69]:
def visualizeGraph(treeList):
    # Create a Graphviz Digraph
    dot = Digraph(comment='Tree')

    # Add nodes and edges to the Digraph
    for node in treeList:
        dot.node(node)

    for node, edges in treeList.items():
        for edge in edges:
            dot.edge(node, edge)

    # Visualize the Digraph
    dot.render('tree', view=True)

'tree.pdf'

In [ ]:
visualizeGraph(adjacencyList)

In [15]:
def checkRight(node : Node, bottom_left : list, top_right : list) -> bool:
    if (node.val[0] > bottom_left[0]) and (node.val[0] < top_right[0]) and (node.val[1] > bottom_left[1]) and (node.val[1] < top_right[1]):
        return True
    else:
        return False

In [16]:
def checkLeft(node : Node, top_left : list, bottom_right : list) -> bool:
    if (node.val[0] > bottom_right[0]) and (node.val[0] < top_left[0]) and (node.val[1] > top_left[1]) and (node.val[1] < bottom_right[1]):
        return True
    else:
        return False

In [17]:
def isLeaf(node : Node) -> bool:
    if node==None:
        return False
    if node.left==None and node.right==None:
        return True
    return False

In [18]:
def ToNode(node : list) -> Node:
    N = Node(node)
    return N

In [19]:
def CountQueryPoints(node : Node, p1 : list, p2 : list) ->int:
    
    NoOfPoints = 0
    if p1[0]==p2[0]:
        print('Collinear points')
        return
    else:
        slope = (p2[1] - p1[1])/(p2[0] - p1[0])
        
        if slope == 0:
            print('Collinear points')
            return
        
        else:
            #RIGHT DIAGONAL
            if isLeaf(node) and slope > 0:
                if checkRight(node, p1, p2):
                    NoOfPoints += 1
                    
            #LEFT DIAGONAL
            elif isLeaf(node) and slope < 0:
                if checkLeft(node, p1, p2):
                    NoOfPoints += 1
                    
            else:
                #RIGHT DIAGONAL
                if slope > 0:
                    NoOfPoints += int(checkRight(node,p1,p2)==True)
                    
                #LEFT DIAGONAL    
                if slope < 0:
                    NoOfPoints += int(checkLeft(node,p1,p2)==True)
                    
                if type(node.left) == list: 
                    Nl = ToNode(node.left)
                else: 
                    Nl = node.left
                if type(node.right) == list: 
                    Nr = ToNode(node.right)
                else: 
                    Nr = node.right
                NoOfPoints += CountQueryPoints(Nl, p1, p2)
                NoOfPoints += CountQueryPoints(Nr, p1, p2)
            
    return NoOfPoints

In [20]:
def Max(root):
    Max, stack = [], []
    cur = root
    while cur or stack:
        while cur:
            stack.append(cur)
            Max.append(cur.weight)
            cur = cur.left
        cur = stack.pop()
        cur = cur.right
        node = stack.pop()
    return max(Max)

In [21]:
def Min(root):
    Max, stack = [], []
    cur = root
    while cur or stack:
        while cur:
            stack.append(cur)
            Max.append(cur.weight)
            cur = cur.left
        cur = stack.pop()
        cur = cur.right
        node = stack.pop()
    return min(Max)

In [22]:
points = [[3,2], [8,4], [6,5]]
weights = [1,2,1]
root = buildKdTree(points,2,weights,0)
print(Max(root))
print(Min(root))

TypeError: buildKdTree() takes from 1 to 3 positional arguments but 4 were given

In [ ]:
points = np.array([(0,1),(3,3),(4,6),(5,5),(10,10),(12,13),(13,14)])

In [ ]:
dfs(root)

In [ ]:
#Case: Right Diagonal
print(CountQueryPoints(root,[-1,-1],[15,15]))

In [ ]:
#Case: Left Diagonal
print(CountQueryPoints(root,[-15,15],[-1,1]))

In [ ]:
#Case: Collinear
print(CountQueryPoints(root,[-1,-1],[15,-1]))

In [ ]:
#Case: Collinear
print(CountQueryPoints(root,[-1,-1],[-1,15]))

In [ ]:
def NNKDTree(queryPoint, root, threshold, noOfPoints):
    listOfNeighbors = []
    return NNKDTreeRec(queryPoint, root, threshold, noOfPoints, listOfNeighbors)

In [ ]:
def squareDistance(p1, p2):
    retValue = abs((p1[0] - p2[0]) ^ 2 - (p1[1] - p2[1]) ^ 2)
    print("p1: ", p1, "p2: ", p2, "print val: ", retValue)
    return retValue

In [ ]:
def NNKDTreeRec(queryPoint, root, threshold, noOfPoints, listOfNeighbors):
    if not root:
        return listOfNeighbors
    else:
        if squareDistance(root.val, queryPoint) < threshold and len(listOfNeighbors) < noOfPoints:
            listOfNeighbors.append(root.val)
        if root.left == None and root.right == None:
            return listOfNeighbors
        else:
            T1, T2 = None, None
            query = queryPoint[0] if root.axis == 0 else queryPoint[1]
            currRoot = root.val[0] if root.axis == 0 else root.val[1]
                
            if query < root.val[0] if root.axis == 0 else root.val[1]:
                T1 = root.left
                T2 = root.right
            else:
                T1 = root.right
                T2 = root.left
            leftList = NNKDTreeRec(queryPoint, T1, threshold, noOfPoints, listOfNeighbors)
            if len(leftList) < noOfPoints and squareDistance(root.val, queryPoint) < threshold:
                rightList = NNKDTreeRec(queryPoint, root.right, threshold, noOfPoints, listOfNeighbors)
            else:
                rightList = NNKDTreeRec(queryPoint, root.right, threshold, noOfPoints, listOfNeighbors)
    return listOfNeighbors

In [ ]:
points, root.val

In [ ]:
NNKDTree([6,4], root, 5, 8)